### 0. Spark Setup

In [1]:
import os
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-sql-kafka-0-10_2.12:3.3.1,org.apache.spark:spark-avro_2.12:3.3.1 pyspark-shell'

In [2]:
from pyspark.sql import SparkSession
import pyspark.sql.types as T
import pyspark.sql.functions as F

spark = SparkSession \
    .builder \
    .appName("Spark-Notebook") \
    .getOrCreate()

:: loading settings :: url = jar:file:/usr/local/lib/python3.10/dist-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
org.apache.spark#spark-avro_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-3bb7814c-8433-474e-b786-d6522d8bec73;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.3.1 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.3.1 in central
	found org.apache.kafka#kafka-clients;2.8.1 in central
	found org.lz4#lz4-java;1.8.0 in central
	found org.xerial.snappy#snappy-java;1.1.8.4 in central
	found org.slf4j#slf4j-api;1.7.32 in central
	found org.apache.hadoop#hadoop-client-runtime;3.3.2 in central
	found org.spark-project.spark#unused;1.0.0 in central
	found org.apache.hadoop#hadoop-client-api;3.3.2 in central
	found commons-logging#commons-logging;1.1.3 in central
	found com.google.code.findbugs#jsr305;3.0.0 in central
	found org.apache.common

23/03/14 14:03:18 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


### 1. Reading from Kafka Stream

through `readStream`

#### 1.1 Raw Kafka Stream

In [3]:
# default for startingOffsets is "latest"
df_kafka_raw = spark \
    .readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "localhost:9092,broker:29092") \
    .option("subscribe", "rides_json") \
    .option("startingOffsets", "earliest") \
    .option("checkpointLocation", "checkpoint") \
    .load()

In [4]:
df_kafka_raw.printSchema()

root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)



#### 1.2 Encoded Kafka Stream

In [5]:
df_kafka_encoded = df_kafka_raw.selectExpr("CAST(key AS STRING)","CAST(value AS STRING)")

In [6]:
df_kafka_encoded.printSchema()

root
 |-- key: string (nullable = true)
 |-- value: string (nullable = true)



#### 1.3 Structure Streaming DataFrame

In [7]:
def parse_ride_from_kafka_message(df_raw, schema):
    """ take a Spark Streaming df and parse value col based on <schema>, return streaming df cols in schema """
    assert df_raw.isStreaming is True, "DataFrame doesn't receive streaming data"

    df = df_raw.selectExpr("CAST(key AS STRING)", "CAST(value AS STRING)")

    # split attributes to nested array in one Column
    col = F.split(df['value'], ', ')

    # expand col to multiple top-level columns
    for idx, field in enumerate(schema):
        df = df.withColumn(field.name, col.getItem(idx).cast(field.dataType))
    return df.select([field.name for field in schema])

In [8]:
ride_schema = T.StructType(
    [
     # T.StructField("vendor_id", T.IntegerType()),
     # T.StructField('tpep_pickup_datetime', T.TimestampType()),
     # T.StructField('tpep_dropoff_datetime', T.TimestampType()),
     T.StructField('pu_location_id', T.IntegerType()),
     # T.StructField("passenger_count", T.IntegerType()),
     # T.StructField("trip_distance", T.FloatType()),
     # T.StructField("payment_type", T.IntegerType()),
     # T.StructField("total_amount", T.FloatType()),
     ])

In [9]:
df_rides = parse_ride_from_kafka_message(df_raw=df_kafka_raw, schema=ride_schema)

In [10]:
df_rides.printSchema()

root
 |-- pu_location_id: integer (nullable = true)



### 2 Sink Operation & Streaming Query

through `writeStream`

---
**Output Sinks**
- File Sink: stores the output to the directory
- Kafka Sink: stores the output to one or more topics in Kafka
- Foreach Sink:
- (for debugging) Console Sink, Memory Sink

Further details can be found in [Output Sinks](https://spark.apache.org/docs/latest/structured-streaming-programming-guide.html#output-sinks)

---
There are three types of **Output Modes**:
- Complete: The whole Result Table will be outputted to the sink after every trigger. This is supported for aggregation queries.
- Append (default): Only new rows are added to the Result Table
- Update: Only updated rows are outputted

[Output Modes](https://spark.apache.org/docs/latest/structured-streaming-programming-guide.html#output-modes) differs based on the set of transformations applied to the streaming data. 

--- 
**Triggers**

The [trigger settings](https://spark.apache.org/docs/latest/structured-streaming-programming-guide.html#triggers) of a streaming query define the timing of streaming data processing. Spark streaming support micro-batch streamings schema and you can select following options based on requirements.

- default-micro-batch-mode
- fixed-interval-micro-batch-mode
- one-time-micro-batch-mode
- available-now-micro-batch-mode


#### Console and Memory Sink

In [11]:
def sink_console(df, output_mode: str = 'complete', processing_time: str = '5 seconds'):
    write_query = df.writeStream \
        .outputMode(output_mode) \
        .trigger(processingTime=processing_time) \
        .format("console") \
        .option("truncate", False) \
        .start()
    return write_query # pyspark.sql.streaming.StreamingQuery

In [18]:
write_query = sink_console(df_rides, output_mode='append')

23/03/14 09:02:25 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-3a4e6633-f7ad-4af0-87b7-924d849e972b. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
23/03/14 09:02:25 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
23/03/14 09:02:26 WARN NetworkClient: [Consumer clientId=consumer-spark-kafka-source-29d8d1ba-5b6c-4aad-a123-5f93fd69bef7--216657764-driver-0-4, groupId=spark-kafka-source-29d8d1ba-5b6c-4aad-a123-5f93fd69bef7--216657764-driver-0] Connection to node -1 (localhost/127.0.0.1:9092) could not be established. Broker may not be available.
23/03/14 09:02:26 WARN NetworkClient: [Consumer clientId=consumer-spark-kafka-source-29d8d1ba-5b6c-4aad-a123-5f93fd69bef7--216657764-driv

In [19]:
def sink_memory(df, query_name, query_template):
    write_query = df \
        .writeStream \
        .queryName(query_name) \
        .format('memory') \
        .start()
    query_str = query_template.format(table_name=query_name)
    query_results = spark.sql(query_str)
    return write_query, query_results

In [20]:
query_name = 'pu_location_count_test1'
query_template = 'select pu_location_id, count(1) from {table_name} group by 1 order by 2 desc'
write_query, df_vendor_id_counts = sink_memory(df=df_rides, query_name=query_name, query_template=query_template)

23/03/14 09:02:27 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-6a625a2b-9cef-4f36-a208-86bd44fe7334. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
23/03/14 09:02:27 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


23/03/14 09:02:27 WARN NetworkClient: [Consumer clientId=consumer-spark-kafka-source-9d046eaa-719b-4034-bc7c-bdbad959fbe4--229402881-driver-0-6, groupId=spark-kafka-source-9d046eaa-719b-4034-bc7c-bdbad959fbe4--229402881-driver-0] Connection to node -1 (localhost/127.0.0.1:9092) could not be established. Broker may not be available.
23/03/14 09:02:27 WARN NetworkClient: [Consumer clientId=consumer-spark-kafka-source-9d046eaa-719b-4034-bc7c-bdbad959fbe4--229402881-driver-0-6, groupId=spark-kafka-source-9d046eaa-719b-4034-bc7c-bdbad959fbe4--229402881-driver-0] Bootstrap broker localhost:9092 (id: -1 rack: null) disconnected


In [ ]:
print(type(write_query)) # pyspark.sql.streaming.StreamingQuery
write_query.status

In [27]:
df_vendor_id_counts.show()

23/03/14 09:14:47 WARN ProcessingTimeExecutor: Current batch is falling behind. The trigger interval is 5000 milliseconds, but spent 12784 milliseconds
23/03/14 09:14:47 WARN ProcessingTimeExecutor: Current batch is falling behind. The trigger interval is 5000 milliseconds, but spent 12846 milliseconds


-------------------------------------------
Batch: 107
-------------------------------------------
-------------------------------------------
Batch: 107
-------------------------------------------
+--------------+
|pu_location_id|
+--------------+
|2             |
|2             |
|2             |
|2             |
|2             |
|2             |
|1             |
|2             |
|2             |
|2             |
|2             |
|2             |
|2             |
|2             |
|2             |
|1             |
|1             |
|2             |
|2             |
|2             |
+--------------+
only showing top 20 rows

+--------------+
|pu_location_id|
+--------------+
|2             |
|2             |
|2             |
|2             |
|2             |
|2             |
|1             |
|2             |
|2             |
|2             |
|2             |
|2             |
|2             |
|2             |
|2             |
|1             |
|1             |
|2             |
|2         

-------------------------------------------
Batch: 108
-------------------------------------------
+--------------+
|pu_location_id|
+--------------+
|2             |
|2             |
|2             |
|2             |
|2             |
|2             |
|2             |
|2             |
|1             |
|1             |
|1             |
|2             |
|2             |
|2             |
|1             |
|2             |
|2             |
|2             |
|2             |
|2             |
+--------------+
only showing top 20 rows

-------------------------------------------
Batch: 108
-------------------------------------------
+--------------+
|pu_location_id|
+--------------+
|2             |
|2             |
|2             |
|2             |
|2             |
|1             |
|1             |
|1             |
|2             |
|2             |
|2             |
|1             |
|2             |
|2             |
|2             |
|2             |
|2             |
|2             |
|2         

23/03/14 09:15:15 WARN TaskSetManager: Stage 715 contains a task of very large size (18324 KiB). The maximum recommended task size is 1000 KiB.


-------------------------------------------
Batch: 110
-------------------------------------------
+--------------+
|pu_location_id|
+--------------+
|2             |
|2             |
|2             |
|2             |
|2             |
|2             |
|2             |
|2             |
|2             |
|2             |
|1             |
|1             |
|2             |
|2             |
|2             |
|2             |
|2             |
|2             |
|2             |
|1             |
+--------------+
only showing top 20 rows

-------------------------------------------
Batch: 110
-------------------------------------------
+--------------+
|pu_location_id|
+--------------+
|2             |
|2             |
|2             |
|2             |
|2             |
|2             |
|2             |
|2             |
|2             |
|2             |
|2             |
|2             |
|2             |
|2             |
|2             |
|2             |
|2             |
|2             |
|2         

23/03/14 09:15:25 WARN ProcessingTimeExecutor: Current batch is falling behind. The trigger interval is 5000 milliseconds, but spent 10209 milliseconds


-------------------------------------------
Batch: 111
-------------------------------------------
+--------------+--------+
|pu_location_id|count(1)|
+--------------+--------+
|          null|  580667|
|             2|  473523|
|             1|   82775|
+--------------+--------+

+--------------+
|pu_location_id|
+--------------+
|2             |
|2             |
|2             |
|2             |
|2             |
|2             |
|2             |
|2             |
|2             |
|2             |
|2             |
|2             |
|1             |
|2             |
|2             |
|2             |
|2             |
|2             |
|2             |
|2             |
+--------------+
only showing top 20 rows

-------------------------------------------
Batch: 111
-------------------------------------------
+--------------+
|pu_location_id|
+--------------+
|2             |
|2             |
|2             |
|2             |
|2             |
|2             |
|2             |
|2             

23/03/14 09:15:58 WARN ProcessingTimeExecutor: Current batch is falling behind. The trigger interval is 5000 milliseconds, but spent 16076 milliseconds


-------------------------------------------
Batch: 116
-------------------------------------------
+--------------+
|pu_location_id|
+--------------+
|2             |
|2             |
|2             |
|2             |
|2             |
|2             |
|2             |
|2             |
|1             |
|2             |
|1             |
|2             |
|2             |
|2             |
|2             |
|2             |
|2             |
|2             |
|2             |
|2             |
+--------------+
only showing top 20 rows

-------------------------------------------
Batch: 115
-------------------------------------------
23/03/14 09:16:01 WARN ProcessingTimeExecutor: Current batch is falling behind. The trigger interval is 5000 milliseconds, but spent 5488 milliseconds
+--------------+
|pu_location_id|
+--------------+
|2             |
|2             |
|2             |
|2             |
|2             |
|2             |
|2             |
|2             |
|1             |
|2           

-------------------------------------------
Batch: 117
-------------------------------------------
+--------------+
|pu_location_id|
+--------------+
|2             |
|1             |
|2             |
|2             |
|2             |
|2             |
|2             |
|2             |
|2             |
|1             |
|2             |
|2             |
|2             |
|2             |
|2             |
|2             |
|2             |
|2             |
|2             |
|2             |
+--------------+
only showing top 20 rows

-------------------------------------------
Batch: 118
-------------------------------------------
23/03/14 09:17:12 WARN ProcessingTimeExecutor: Current batch is falling behind. The trigger interval is 5000 milliseconds, but spent 36187 milliseconds
+--------------+
|pu_location_id|
+--------------+
|2             |
|2             |
|2             |
|2             |
|2             |
|2             |
|2             |
|2             |
|2             |
|2          

In [17]:
write_query.stop()

23/03/14 09:02:12 ERROR WriteToDataSourceV2Exec: Data source write support org.apache.spark.sql.execution.streaming.sources.MicroBatchWrite@764373cc is aborting.
23/03/14 09:02:12 ERROR WriteToDataSourceV2Exec: Data source write support org.apache.spark.sql.execution.streaming.sources.MicroBatchWrite@764373cc aborted.
23/03/14 09:02:12 ERROR Utils: Aborting task
org.apache.spark.TaskKilledException
	at org.apache.spark.TaskContextImpl.killTaskIfInterrupted(TaskContextImpl.scala:217)
	at org.apache.spark.InterruptibleIterator.hasNext(InterruptibleIterator.scala:36)
	at scala.collection.Iterator$$anon$10.hasNext(Iterator.scala:460)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.processNext(Unknown Source)
	at org.apache.spark.sql.execution.BufferedRowIterator.hasNext(BufferedRowIterator.java:43)
	at org.apache.spark.sql.execution.WholeStageCodegenExec$$anon$1.hasNext(WholeStageCodegenExec.scala:760)
	at org.apache.spark.sql.execution.dataso

-------------------------------------------
Batch: 0
-------------------------------------------


#### Kafka Sink

To write stream results to `kafka-topic`, the stream dataframe has at least a column with name `value`.

Therefore before starting `writeStream` in kafka format, dataframe needs to be updated accordingly.

More information regarding kafka sink expected data structure [here](https://spark.apache.org/docs/latest/structured-streaming-kafka-integration.html#writing-data-to-kafka)


In [7]:
def prepare_dataframe_to_kafka_sink(df, value_columns, key_column=None):
    columns = df.columns
    df = df.withColumn("value", F.concat_ws(', ',*value_columns))    
    if key_column:
        df = df.withColumnRenamed(key_column,"key")
        df = df.withColumn("key",df.key.cast('string'))
    return df.select(['key', 'value'])
    
def sink_kafka(df, topic, output_mode='append'):
    write_query = df.writeStream \
        .format("kafka") \
        .option("kafka.bootstrap.servers", "localhost:9092,broker:29092") \
        .outputMode(output_mode) \
        .option("topic", topic) \
        .option("checkpointLocation", "checkpoint") \
        .start()
    return write_query

def op_groupby(df, column_names):
    df_aggregation = df.groupBy(column_names).count()
    return df_aggregation

def op_windowed_groupby(df, window_duration, slide_duration):
    df_windowed_aggregation = df.groupBy(
        F.window(timeColumn=df.tpep_pickup_datetime, windowDuration=window_duration, slideDuration=slide_duration),
        df.pu_location_id
    ).count()
    return df_windowed_aggregation